# Download emissions data using a UKEMS URL

The UKEMS download page (https://dukems-app.datalabs.ceh.ac.uk/app/download) allows you to download emissions data and also  copy the download URL for your own use.  This URL specifies which pollutant you want, and also additional filters such as Snap Sector, region of interest and date range.  A full description is at https://dukems-app.datalabs.ceh.ac.uk/app/apihelp.

This notebook shows how to use this URL to simply download the data and display it.  It doesn't necessarily demonstrate best practice, but is written to be simple and help you think about how you might automate and integrate it into your own code.


### Install dependencies and import libraries

In [ ]:
!pip install xarray
!pip install netCDF4

import requests
import xarray as xr
import matplotlib.pyplot as plt
import io
import re
import os
import warnings

# For clarity, turn off warnings
warnings.filterwarnings('ignore')

### Open emissions data using a UK-EMS URL

A UK-EMS download URL can obtained from https://dukems-app.datalabs.ceh.ac.uk/app/download.  The following code block will prompt you for a URL and then save the data it references to a file using the name found in the request header and open it to display its metadata.

Here is an example UK-EMS download URL: https://dukems-app.datalabs.ceh.ac.uk/api/download/dataset/5/7/0/0?startYear=2020&endYear=2030&y2=238954&y1=114353&x2=633171&x1=461485.

In [ ]:
url = input("Enter a UKEMS emissions download URL")

r = requests.get(url, verify=False, allow_redirects=True)
filename = re.findall('filename=(.+)', r.headers.get('content-disposition'))[0]
open(filename, 'wb').write(r.content)
print(f'Downloaded the file {filename} ({os.path.getsize(filename)} bytes)')
with xr.open_dataset(filename) as ds:
    print(ds)


### Plot a time series for a location in the downloaded data

The downloaded data is for a region around central London.  The centre of London has the British National Grid coordinate: 530034,180381.  Putting a 1km box around this coordinate, will restrict the data to nearest 1km square in the data.  Its annual emissions can then be plotted.

In [ ]:
    london_x = 530034
    london_y = 180381
    dataForCentralLondon = ds.sel(x=slice(london_x-500,london_x+500), y=slice(london_y+500,london_y-500))
    emissions = dataForCentralLondon['emissions_data'].data.flatten()
    years =dataForCentralLondon['Time'].data

    print('Time series for central London:')
    plt.scatter(years, emissions)
    plt.show()


### View some data

In [ ]:
ds.loc[dict(Time=2023)]['emissions_data'].plot()